<a href="https://colab.research.google.com/github/shravankoninti/AV/blob/master/AV_network_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import pickle

In [0]:

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#train.csv
# https://drive.google.com/open?id=1pRddOtR2o_1Ywa_tpNglxsKv5eA5ROgk

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1pRddOtR2o_1Ywa_tpNglxsKv5eA5ROgk' #https://drive.google.com/open?id=1BfqCYaQ21TTWWWe2dCo0riuD-uKL_Cct
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train.zip')

In [4]:
!ls

adc.json			    test.csv
new_dict_source.pkl		    train.csv
new_dict_target.pkl		    train.zip
sample_data			    user_features.csv
sample_submission_only_headers.csv  user_features_df_dict.pkl


In [0]:
#train.csv
# https://drive.google.com/open?id=1Az5lmh-XGkNXkO_k_3IZTmLg_Sf7teJb

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1Az5lmh-XGkNXkO_k_3IZTmLg_Sf7teJb' #https://drive.google.com/open?id=1BfqCYaQ21TTWWWe2dCo0riuD-uKL_Cct
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test.csv')

In [6]:
!ls

adc.json			    test.csv
new_dict_source.pkl		    train.csv
new_dict_target.pkl		    train.zip
sample_data			    user_features.csv
sample_submission_only_headers.csv  user_features_df_dict.pkl


In [0]:
# https://drive.google.com/open?id=1RsyDZfkcLNib_YU3Bp4bRdD4FjqrmNpk
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1RsyDZfkcLNib_YU3Bp4bRdD4FjqrmNpk' 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('sample_submission_only_headers.csv')

In [8]:
!ls

adc.json			    test.csv
new_dict_source.pkl		    train.csv
new_dict_target.pkl		    train.zip
sample_data			    user_features.csv
sample_submission_only_headers.csv  user_features_df_dict.pkl


In [9]:
!unzip train.zip

Archive:  train.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace user_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [10]:
!ls

adc.json			    test.csv
new_dict_source.pkl		    train.csv
new_dict_target.pkl		    train.zip
sample_data			    user_features.csv
sample_submission_only_headers.csv  user_features_df_dict.pkl


In [11]:
submission_df = pd.read_csv('sample_submission_only_headers.csv')
submission_df.head()

,id,is_chat


In [12]:
train_df = pd.read_csv('train.csv')
train_df.head()

,node1_id,node2_id,is_chat
0,8446602,6636127,0
1,1430102,7433949,0
2,2803017,8372333,0
3,4529348,894645,0
4,5096572,4211638,0


In [13]:
train_df['is_chat'].value_counts(normalize = True)

0    0.968133
1    0.031867
Name: is_chat, dtype: float64

In [14]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,node1_id,node2_id
0,1,7107094,8010772
1,2,7995251,2805801
2,3,2804693,8059549
3,4,4812472,7332370
4,5,5009985,4511909


In [15]:
test_df.shape

(11776968, 3)

In [16]:
user_features_df = pd.read_csv('user_features.csv')
user_features_df.head()

,node_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,2,14,14,14,12,12,12,7,7,7,0,0,0,15
1,3,31,9,7,31,16,12,31,15,12,31,15,12,8
2,4,0,0,0,0,0,0,0,0,0,0,0,0,7
3,5,31,4,1,31,7,1,31,9,1,31,9,0,15
4,6,31,27,20,31,24,14,31,20,10,31,20,5,7


In [17]:
!pip install networkx

In [18]:
user_features_df.columns

Index(['node_id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13'],
      dtype='object')

In [0]:
# # for each training example we need to compute features
# # in this baseline we will train the model on only 5% of the training set

# # randomly select 5% of training set
# to_keep = random.sample(range(len(train_df)), k=int(round(len(train_df)*0.05)))
# # training_set_reduced = [train_df[i] for i in to_keep]

In [20]:
training_set_reduced = train_df.sample(frac=0.0005)
training_set_reduced.head(3)

,node1_id,node2_id,is_chat
59103891,7138399,4780120,0
60189937,52249,2583694,0
3170375,1216239,1807852,0


In [21]:
print(training_set_reduced.shape)
print(training_set_reduced['is_chat'].value_counts())

(35331, 3)
0    34215
1     1116
Name: is_chat, dtype: int64


In [0]:
# print(training_set_reduced.iloc[0][0])
# print(training_set_reduced.iloc[0][1])

# source = training_set_reduced.iloc[0][0]
# target = training_set_reduced.iloc[0][1]


In [0]:
# source_list = list(training_set_reduced['node1_id'])
# target_list = list(training_set_reduced['node2_id'])

In [0]:
# user_features_df_list = list(user_features_df.values)

In [0]:
# write python dict to a file

# user_features_df_dict = user_features_df.set_index('node_id').T.to_dict('list')

# output = open('user_features_df_dict.pkl', 'wb')
# pickle.dump(user_features_df_dict, output)
# output.close()

In [0]:
# file_id = '12yMza2_i50Wxm05ZRk8nGu8HtAw5SS13'
# # https://drive.google.com/open?id=12yMza2_i50Wxm05ZRk8nGu8HtAw5SS13
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('user_features_df_dict.pkl')

In [0]:
# # read python dict back from the file
# pkl_file = open('user_features_df_dict.pkl', 'rb')
# user_features_df_dict = pickle.load(pkl_file)
# pkl_file.close()

In [0]:
# print(len(user_features_df_dict))

In [0]:
# print(len(source_list))
# print(len(target_list))

In [0]:
# %%timeit
# new_dict_source = dict((k, v) for k, v in user_features_df_dict.items() if k in source_list)
# new_dict_target = dict((k, v) for k, v in user_features_df_dict.items() if k in target_list)

# print(len(new_dict_source))
# print(len(new_dict_target))

In [0]:
# output = open('new_dict_source.pkl', 'wb')
# pickle.dump(new_dict_source, output)
# output.close()


# output = open('new_dict_target.pkl', 'wb')
# pickle.dump(new_dict_target, output)
# output.close()

In [0]:
# file_id = '1XhY27_XUE1Q1harIyB_jEzGqNCBBaqx_'
# # https://drive.google.com/open?id=1XhY27_XUE1Q1harIyB_jEzGqNCBBaqx_
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('new_dict_source.pkl')


# # read python dict back from the file
# pkl_file = open('new_dict_source.pkl', 'rb')
# new_dict_source = pickle.load(pkl_file)
# pkl_file.close()

# file_id = '1OaQPN4416pNu3h_40iF7Y47tOy32FKZq'
# # https://drive.google.com/open?id=1OaQPN4416pNu3h_40iF7Y47tOy32FKZq
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('new_dict_target.pkl')


# # read python dict back from the file
# pkl_file = open('new_dict_target.pkl', 'rb')
# new_dict_target = pickle.load(pkl_file)
# pkl_file.close()

In [0]:
# print(len(new_dict_target))
# print(len(new_dict_source))

In [0]:
# ls1 = list(new_dict_source.keys())
# print(len(ls1))

In [0]:
# ls2 = list(new_dict_target.keys())
# print(len(ls2))

In [35]:
test_df.head(4)

,id,node1_id,node2_id
0,1,7107094,8010772
1,2,7995251,2805801
2,3,2804693,8059549
3,4,4812472,7332370


In [0]:
# test_source_node_features = pd.merge(test_df[['node1_id']], user_features_df, how='inner', left_on='node1_id', right_on='node_id')
# print(test_source_node_features.shape)
# print(test_source_node_features.head(4))

# # dropping ALL duplicte values 
# test_source_node_features.drop_duplicates(subset ="node1_id", keep = False, inplace = True) 
# print(test_source_node_features.shape)
# print(test_source_node_features.head(4))

# test_source_node_features.drop(['node_id'], axis=1, inplace= True)

# print(test_source_node_features.shape)
# print(test_source_node_features.head(4))

In [0]:
# test_target_node_features = pd.merge(test_df[['node2_id']], user_features_df, how='inner', left_on='node2_id', right_on='node_id')
# print(test_target_node_features.shape)
# print(test_target_node_features.head(4))

# # dropping ALL duplicte values 
# test_target_node_features.drop_duplicates(subset ="node2_id", keep = False, inplace = True) 
# print(test_target_node_features.shape)
# print(test_target_node_features.head(4))

# test_target_node_features.drop(['node_id'], axis=1, inplace= True)

# print(test_target_node_features.shape)
# print(test_target_node_features.head(4))

In [38]:
s1 = pd.merge(test_df, user_features_df, how='left', left_on='node1_id', right_on='node_id')
print(s1.shape)
print(s1.head(4))

s1.drop(['node_id'],axis=1,inplace= True)
s1.rename(columns={'f1': 'f1_node1', 'f2': 'f2_node1', 'f3': 'f3_node1',
                   'f4': 'f4_node1', 'f5': 'f5_node1', 'f6': 'f6_node1',
                   'f7': 'f7_node1', 'f8': 'f8_node1', 'f9': 'f9_node1',
                   'f10': 'f10_node1', 'f11': 'f11_node1', 'f12': 'f12_node1',
                   'f13': 'f13_node1'                  
                  }, inplace=True)



print(s1.shape)
print(s1.head(4))

(11776968, 17)
   id  node1_id  node2_id  node_id  f1  f2  f3  f4  f5  f6  f7  f8  f9  f10  \
0   1   7107094   8010772  7107094  31   6   1  31   6   1  31   8   1   31   
1   2   7995251   2805801  7995251   0   0   0   0   0   0   0   0   0    6   
2   3   2804693   8059549  2804693   1   1   1   0   0   0   0   0   0    0   
3   4   4812472   7332370  4812472  14   5   1  12   4   1   6   3   1    0   

   f11  f12  f13  
0    9    1   15  
1    0    0    7  
2    0    0   15  
3    0    0   15  
(11776968, 16)
   id  node1_id  node2_id  f1_node1  f2_node1  f3_node1  f4_node1  f5_node1  \
0   1   7107094   8010772        31         6         1        31         6   
1   2   7995251   2805801         0         0         0         0         0   
2   3   2804693   8059549         1         1         1         0         0   
3   4   4812472   7332370        14         5         1        12         4   

   f6_node1  f7_node1  f8_node1  f9_node1  f10_node1  f11_node1  f12_node1  \
0    

In [39]:
s2 = pd.merge(s1, user_features_df, how='left', left_on='node2_id', right_on='node_id')
print(s2.shape)
print(s2.head(4))



(11776968, 30)
   id  node1_id  node2_id  f1_node1  f2_node1  f3_node1  f4_node1  f5_node1  \
0   1   7107094   8010772        31         6         1        31         6   
1   2   7995251   2805801         0         0         0         0         0   
2   3   2804693   8059549         1         1         1         0         0   
3   4   4812472   7332370        14         5         1        12         4   

   f6_node1  f7_node1 ...   f4  f5  f6  f7  f8  f9  f10  f11  f12  f13  
0         1        31 ...   31  24   3  31  25   3   31   26    4   15  
1         0         0 ...    0   0   0   0   0   0    0    0    0    7  
2         0         0 ...    0   0   0   0   0   0    0    0    0   15  
3         1         6 ...    0   0   0   1   1   0    1    1    0   15  

[4 rows x 30 columns]


In [40]:
s2.columns

Index(['id', 'node1_id', 'node2_id', 'f1_node1', 'f2_node1', 'f3_node1',
       'f4_node1', 'f5_node1', 'f6_node1', 'f7_node1', 'f8_node1', 'f9_node1',
       'f10_node1', 'f11_node1', 'f12_node1', 'f13_node1', 'node_id', 'f1',
       'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12',
       'f13'],
      dtype='object')

In [0]:
del [[s1]]
import gc
gc.collect()
s1=pd.DataFrame()




In [0]:
s2['f1_sum'] = s2['f1_node1']+s2['f1']
s2['f2_sum'] = s2['f2_node1']+s2['f2']
s2['f3_sum'] = s2['f3_node1']+s2['f3']
s2['f4_sum'] = s2['f4_node1']+s2['f4']
s2['f5_sum'] = s2['f5_node1']+s2['f5']
s2['f6_sum'] = s2['f6_node1']+s2['f6']
s2['f7_sum'] = s2['f7_node1']+s2['f7']
s2['f8_sum'] = s2['f8_node1']+s2['f8']
s2['f9_sum'] = s2['f9_node1']+s2['f9']
s2['f10_sum'] = s2['f10_node1']+s2['f10']
s2['f11_sum'] = s2['f11_node1']+s2['f11']
s2['f12_sum'] = s2['f12_node1']+s2['f12']
s2['f13_sum'] = s2['f13_node1']+s2['f13']
s2.drop(['node1_id', 'node2_id', 'f1_node1', 'f2_node1', 'f3_node1',
       'f4_node1', 'f5_node1', 'f6_node1', 'f7_node1', 'f8_node1', 'f9_node1',
       'f10_node1', 'f11_node1', 'f12_node1', 'f13_node1', 'node_id', 'f1',
       'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12',
       'f13'], axis=1, inplace=True)
print(s2.shape)
print(s2.head(3))


In [0]:
# s2.drop(['node_id'],axis=1,inplace= True)
# s2.rename(columns={'f1': 'f1_node2', 'f2': 'f2_node2', 'f3': 'f3_node2',
#                    'f4': 'f4_node2', 'f5': 'f5_node2', 'f6': 'f6_node2',
#                    'f7': 'f7_node2', 'f8': 'f8_node2', 'f9': 'f9_node2',
#                    'f10': 'f10_node2', 'f11': 'f11_node2', 'f12': 'f12_node2',
#                    'f13': 'f13_node2'                  
#                   }, inplace=True)


# print(s2.shape)
# print(s2.head(4))

## Test dictionary mapping


In [1]:
source_test_list = list(test_df['node1_id'])
target_test_list = list(test_df['node2_id'])

new_dict_test_source = dict((k, v) for k, v in user_features_df_dict.items() if k in source_test_list)
new_dict_test_target = dict((k, v) for k, v in user_features_df_dict.items() if k in target_test_list)

print(len(new_dict_test_source))
print(len(new_dict_test_target))

output = open('new_dict_test_source.pkl', 'wb')
pickle.dump(new_dict_test_source, output)
output.close()

from google.colab import files
files.download('new_dict_test_source.pkl')


output = open('new_dict_test_target.pkl', 'wb')
pickle.dump(new_dict_test_target, output)
output.close()

from google.colab import files
files.download('new_dict_test_target.pkl')

# # read python dict back from the file
# pkl_file = open('new_dict_test_source.pkl', 'rb')
# new_dict_test_source = pickle.load(pkl_file)
# pkl_file.close()




# # read python dict back from the file
# pkl_file = open('new_dict_test_target.pkl', 'rb')
# new_dict_test_target = pickle.load(pkl_file)
# pkl_file.close()

NameError: ignored

In [0]:
# def add_source_target(data, user_feat):  
#   for i in range(data.shape[0]):
#     source = data.iloc[i][0]
#     target = data.iloc[i][1]
    
#     for j in range(user_feat.shape[0]):
#       if (user_feat['node_id'][j] == source):
#         user_feat['tag'][j] = 'source'
#       elif (user_feat['node_id'][j] == target):
#         user_feat['tag'][j] = 'target'
#       else:
#         user_feat['tag'][j] = 'None'++

#   return user_feat       

In [0]:
# new_data = add_source_target(training_set_reduced, user_features_df)
# new_data.to_csv('new_data.csv')

In [0]:
# f1_diff, f2_diff, f3_diff, f4_diff, f5_diff, f6_diff, f7_diff,f8_diff, f9_diff, f10_diff, f11_diff, f12_diff, f13_diff = ([] for i in range(13)) 

# f1_sum, f2_sum, f3_sum, f4_sum, f5_sum, f6_sum, f7_sum,f8_sum, f9_sum, f10_sum, f11_sum, f12_sum, f13_sum = ([] for i in range(13)) 


# counter = 0
# for i in range(training_set_reduced.shape[0]):
#     source = training_set_reduced.iloc[i][0]
#     target = training_set_reduced.iloc[i][1]
    
#     source_info = [element for element in user_features_df_list if element[0]==source][0]

#     target_info = [element for element in user_features_df_list if element[0]==target][0]
  
  
#     f1_sum.append((source_info[1]) + (target_info[1]))
#     f2_sum.append((source_info[2]) + (target_info[2]))
#     f3_sum.append((source_info[3]) + (target_info[3]))
#     f4_sum.append((source_info[4]) + (target_info[4]))
#     f5_sum.append((source_info[5]) + (target_info[5]))
#     f6_sum.append((source_info[6]) + (target_info[6]))
#     f7_sum.append((source_info[7]) + (target_info[7]))
#     f8_sum.append((source_info[8]) + (target_info[8]))
#     f9_sum.append((source_info[9]) + (target_info[9]))
#     f10_sum.append((source_info[10]) + (target_info[10]))
#     f11_sum.append((source_info[11]) + (target_info[11]))
#     f12_sum.append((source_info[12]) + (target_info[12]))
#     f13_sum.append((source_info[13]) + (target_info[13]))
    
   
#     counter += 1
    
#     if counter % 1000 == True:
#         print(counter, "training examples processsed")


In [0]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
training_features = np.array([f1_sum, f2_sum, f3_sum, f4_sum, f5_sum, f6_sum, f7_sum,f8_sum, f9_sum, f10_sum, f11_sum, f12_sum, f13_sum]).T
 

# scale
training_features = preprocessing.scale(training_features)

#save
training_features.to_csv('training_features.csv')
from google.colab import files
files.download('training_features.csv')

# convert labels into integers then into column array
training_set_reduced_list = list(training_set_reduced.values)

labels = [int(element[2]) for element in training_set_reduced_list]
labels = list(labels)
labels_array = np.array(labels)


#labelssave
labels_array.to_csv('labels_array.csv')
from google.colab import files
files.download('labels_array.csv')


# initialize basic SVM
classifier = svm.LinearSVC()

# train
classifier.fit(training_features, labels_array)

In [0]:
test_df.shape

In [0]:
# # test
# # we need to compute the features for the testing set

# f1_diff_test, f2_diff_test, f3_diff_test, f4_diff_test, f5_diff_test, f6_diff_test, f7_diff_test,f8_diff_test, f9_diff_test, f10_diff_test, f11_diff_test, f12_diff_test,f13_diff_test  = ([] for i in range(13)) 

# f1_sum_test, f2_sum_test, f3_sum_test, f4_sum_test, f5_sum_test, f6_sum_test, f7_sum_test,f8_sum_test, f9_sum_test, f10_sum_test, f11_sum_test, f12_sum_test, f13_sum_test = ([] for i in range(13)) 



# counter = 0
# for i in range(test_df.shape[0]):
#     source = test_df.iloc[i][0]
#     target = test_df.iloc[i][1]
    
#     source_info = [element for element in user_features_df_list if element[0]==source][0]

#     target_info = [element for element in user_features_df_list if element[0]==target][0]
  
  
#     f1_sum_test.append((source_info[1]) + (target_info[1]))
#     f2_sum_test.append((source_info[2]) + (target_info[2]))
#     f3_sum_test.append((source_info[3]) + (target_info[3]))
#     f4_sum_test.append((source_info[4]) + (target_info[4]))
#     f5_sum_test.append((source_info[5]) + (target_info[5]))
#     f6_sum_test.append((source_info[6]) + (target_info[6]))
#     f7_sum_test.append((source_info[7]) + (target_info[7]))
#     f8_sum_test.append((source_info[8]) + (target_info[8]))
#     f9_sum_test.append((source_info[9]) + (target_info[9]))
#     f10_sum_test.append((source_info[10]) + (target_info[10]))
#     f11_sum_test.append((source_info[11]) + (target_info[11]))
#     f12_sum_test.append((source_info[12]) + (target_info[12]))
#     f13_sum_test.append((source_info[13]) + (target_info[13]))
    
   
#     counter += 1
    
#     if counter % 1000 == True:
#         print(counter, "training examples processsed")



In [0]:
# # convert list of lists into array
# # documents as rows, unique words as columns (i.e., example as rows, features as columns)
# testing_features = np.array([f1_sum_test, f2_sum_test, f3_sum_test, f4_sum_test, f5_sum_test, f6_sum_test, f7_sum_test,f8_sum_test, f9_sum_test, f10_sum_test, f11_sum_test, f12_sum_test, f13_sum_test]).T
# # scale
# testing_features = preprocessing.scale(testing_features)

# # issue predictions
# predictions_SVM = list(classifier.predict(testing_features))

# submission_df['id'] = test_df['id']
# submission_df['is_chat'] = pd.DataFrame(predictions_SVM)
# submission_df.head(5)



# # # write predictions to .csv file suitable for Kaggle (just make sure to add the column names)
# # predictions_SVM = zip(range(len(testing_set)), predictions_SVM)

# # with open("improved_predictions.csv","wb") as pred1:
# #     csv_out = csv.writer(pred1)
# #     for row in predictions_SVM:
# #         csv_out.writerow(row)

In [0]:
# submission_df.to_csv('submission_df.csv')

### To create a zip file in python all you need to do is

In [0]:
# import zipfile

# restoreZip = zipfile.ZipFile("submission_df.zip", "w")
# restoreZip.write(../content/sample_data, logicalPathOfFileInZip)
# restoreZip.close()

In [0]:
print("Random float number between 2.5 and 22.5 with two decimal places is ", round(random.uniform(0,1), 3))

In [0]:
###################
# random baseline #
###################

# random_predictions = np.random.choice([0, 1], size=len(testing_set))
# random_predictions = zip(range(len(testing_set)),random_predictions)
pred = []

for i in range(test_df.shape[0]):
  pred.append(round(random.uniform(0,1), 3))

pred_df = pd.DataFrame(pred)
print(len(pred))

In [0]:
print(test_df.shape)

In [0]:
submission_df['id'] = test_df['id']
submission_df['is_chat'] = pd.DataFrame(pred_df)
submission_df.head(5)

In [0]:
submission_df.to_csv('submission_df.csv')

In [0]:
# from shutil import make_archive

# import shutil
# from zipfile import ZipFile
 
# # creates a ZIP archive based on a provided directory
# shutil.make_archive('submission_df', 'zip', '/content/')
 
# # writes specified files to a ZIP archive
# zip_obj = zipfile.ZipFile('archive.zip', 'w')
# zip_obj.write('submission_df.csv')
# zip_obj.close()
